In [ ]:
!pip install pyspark

In [ ]:
!pip install findspark

In [ ]:
import findspark
findspark.init()
findspark.find()

'/usr/local/lib/python3.10/dist-packages/pyspark'

In [ ]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc=SparkContext.getOrCreate()
spark=SparkSession(sc)
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [ ]:
df = spark.read.csv('Movies.csv',inferSchema=True,header=True)

df.show()


+----+------+--------------------+-------+--------------------+-----------------+--------------------+----------+------+-------------------+
|Year|Length|               Title|  Genre|               Actor|          Actress|            Director|Popularity|Awards|              Image|
+----+------+--------------------+-------+--------------------+-----------------+--------------------+----------+------+-------------------+
|1990|   111|Tie Me Up! Tie Me...| Comedy|     BanderasAntonio|    AbrilVictoria|      Almod�varPedro|        68|    No|   NicholasCage.png|
|1991|   113|          High Heels| Comedy|          Bos�Miguel|    AbrilVictoria|      Almod�varPedro|        68|    No|   NicholasCage.png|
|1983|   104|        Dead ZoneThe| Horror|   WalkenChristopher|      AdamsBrooke|     CronenbergDavid|        79|    No|   NicholasCage.png|
|1979|   122|                Cuba| Action|         ConnerySean|      AdamsBrooke|       LesterRichard|         6|    No|    seanConnery.png|
|1978|    94|

## Movies.csv.xlsx was converted to [Movies.csv](https://drive.google.com/file/d/1Ei0T3CnzkCUCvT1t5VgzQ3UdwrxagvAk/view?usp=sharing)

In [ ]:
# 1. Find the title, year, and director of action films that won an award.
one = df.filter((df['Genre']=='Action') & (df['Awards']=='Yes'))
one.select(df.Title,df.Year,df.Director).show()

+-----+----+--------+
|Title|Year|Director|
+-----+----+--------+
+-----+----+--------+



In [ ]:
# 2. For each award-winning actor, find the movies he acted it. Print the names of the movies and the director of the
# movie.
from pyspark.sql.functions import collect_list, struct
two = df.filter(df.Awards == 'Yes')
result = two.groupBy(df.Actor).agg(collect_list(struct('Title', 'Director'))).alias('Movies')
result.show(truncate=False)

+-----------------+------------------------------------------------------------------------------------------------------------------+
|Actor            |collect_list(struct(Title, Director))                                                                             |
+-----------------+------------------------------------------------------------------------------------------------------------------+
|LintDerek De     |[{AssaultThe, RademakersFons}]                                                                                    |
|LancasterBurt    |[{Airport, SeatonGeorge}, {Come BackLittle Sheba, MannDaniel}]                                                    |
|BridgesBeau      |[{Norma Rae, RittMartin}]                                                                                         |
|CapolicchioLino  |[{Garden of the Finzi-ContinisThe, De SicaVittorio}]                                                              |
|LoneJohn         |[{Last EmperorThe, BertolucciBernard

In [ ]:
# 3. Find the top 10 most popular movies that did not win an award.
three = df.filter(df.Awards != 'Yes').orderBy(df.Popularity,ascending=False)
three.select('Title').limit(10).show()

+--------------------+
|               Title|
+--------------------+
|        Five Corners|
|Ballad of Narayam...|
|         Let It Ride|
|        Final Notice|
|      New Year's Day|
| Guilty by Suspicion|
|   Fellini Satyricon|
|           Raw Nerve|
|     Time MachineThe|
| Long Voyage HomeThe|
+--------------------+



In [ ]:
# 4. Find the 10 least popular movies that were released before 1980.
four = df.filter(df.Year < 1980).orderBy(df.Popularity)
four.select('Title').limit(10).show()


+------------------+
|             Title|
+------------------+
|   White Lightning|
|      Drop KickThe|
|      Desert Rider|
| Bank on the Stars|
|           Shalako|
|           Airport|
|     Anna Christie|
|Shout at the Devil|
| Tales of Tomorrow|
|         Holocaust|
+------------------+



In [ ]:
# 5. Find the average length of the movies of each genre.
from pyspark.sql.functions import avg
df.groupBy('Genre').agg(avg('Length')).show()

+---------------+------------------+
|          Genre|       avg(Length)|
+---------------+------------------+
|          Crime|              66.0|
|        Romance|             127.0|
|      Adventure|             119.0|
|           NULL|             120.5|
|          Drama|113.30455259026688|
|            War|         116.90625|
|        Fantasy|             102.0|
|        Mystery|103.00990099009901|
|          Music|100.48780487804878|
|Science Fiction|106.47368421052632|
|         Horror| 93.92727272727272|
|          Short|              40.0|
|        Western|  93.0091743119266|
|         Comedy| 96.50540540540541|
|         Action|             104.5|
|       Westerns|             124.8|
+---------------+------------------+



In [ ]:
from pyspark.sql.functions import col # col was necessary to read new column count
# 6. Find the actor and actress pair who has acted in more than three Comedies together.
six = df.filter(df.Genre == 'Comedy').groupBy('Actor', 'Actress').count().where(col('count') > 3).show()

+-------------+----------------+-----+
|        Actor|         Actress|count|
+-------------+----------------+-----+
|ChapmanGraham|            NULL|    6|
| TracySpencer|HepburnKatharine|    6|
|  MurphyEddie|            NULL|    4|
|   CleeseJohn|            NULL|    4|
| SellersPeter|            NULL|   11|
|   AllenWoody|     KeatonDiane|    5|
|  MartinSteve|            NULL|    4|
|WilliamsRobin|            NULL|    5|
+-------------+----------------+-----+



In [ ]:
# 7. Find the names of actors who acted in movies of both ‘Comedy’ and ‘Drama’ Genre.
actors_comedy = df.filter(df.Genre == 'Comedy').select('Actor').distinct()
actors_drama = df.filter(df.Genre == 'Drama').select('Actor').distinct()
actors_comedy.intersect(actors_drama).show()

+-----------------+
|            Actor|
+-----------------+
|      WillisBruce|
|      IronsJeremy|
|    EastwoodClint|
|      ConnerySean|
|     TracySpencer|
|       NelsonJudd|
|      AielloDanny|
|      MooreDudley|
|       QuinnAidan|
|   AdolphsonEdvin|
|       FinchPeter|
|     BrandoMarlon|
|    HopkinsHarold|
|Bj�rnstrandGunnar|
|       NewmanPaul|
|     BeattyWarren|
|     BoyerCharles|
|        CaanJames|
|  HowellC. Thomas|
|     SheenCharlie|
+-----------------+
only showing top 20 rows



In [ ]:
# 8. Find the names of actors who acted in movies of both ‘Comedy’ or ‘Drama’ Genre.
actors_comedy.union(actors_drama).distinct().show()

+-------------+
|        Actor|
+-------------+
|KeatonMichael|
|  WillisBruce|
| JourdanLouis|
|ModineMatthew|
|  JaglomHenry|
|  BakulaScott|
|  DoranJohnny|
|     LeeKarla|
|  RobertsEric|
|  BelushiJohn|
|     WaitsTom|
|     CryerJon|
|  IronsJeremy|
|  MoranisRick|
|EastwoodClint|
|  ConnerySean|
|  BegleyEdJr.|
| CassidyDavid|
|   PiscopoJoe|
| FeldmanMarty|
+-------------+
only showing top 20 rows



In [ ]:
# 9. Find the names of actors who did not act in any ‘Comedy’.
all_actors = df.select('Actor').distinct()
all_actors.subtract(actors_comedy).show()


+---------------+
|          Actor|
+---------------+
|   CottenJoseph|
|       BrownTom|
|     DillonMatt|
| ShimuraTakashi|
|   LintDerek De|
|  LancasterBurt|
|    RomeroCesar|
|  StockwellDean|
|    UrichRobert|
|       DavisGuy|
|    BridgesBeau|
|    KattWilliam|
|  EnglundRobert|
|      PriceMarc|
|CapolicchioLino|
|     FondaPeter|
| TownsendRobert|
|  ChesneyArthur|
|      BoyceAlan|
|       LoneJohn|
+---------------+
only showing top 20 rows



In [ ]:
# 10. Find each actor, find the mean, max, and min ranking of his movies.
from pyspark.sql.functions import mean, max, min
df.groupBy('Actor').agg(mean('Popularity'), max('Popularity'), min('Popularity')).show()

+--------------+------------------+---------------+---------------+
|         Actor|   avg(Popularity)|max(Popularity)|min(Popularity)|
+--------------+------------------+---------------+---------------+
|  CottenJoseph|              58.0|             74|             32|
|      BrownTom|              77.0|             77|             77|
|    DillonMatt|               7.5|             11|              4|
| KeatonMichael|              59.0|             59|             59|
|ShimuraTakashi|              36.0|             36|             36|
|  LintDerek De|              71.0|             71|             71|
|   WillisBruce|              48.0|             76|              7|
| LancasterBurt|40.291666666666664|             84|              0|
|   RomeroCesar|              78.0|             78|             78|
| StockwellDean|              70.0|             70|             70|
|  JourdanLouis|              70.0|             70|             70|
| ModineMatthew|              26.5|             

In [ ]:
from pyspark.sql.functions import floor, col

# 11. List the number of movies released in each decade starting from the 1960’s.
ele = df.withColumn('Decade', (floor(df['Year']/10).cast('integer')*10))
ele = ele.filter(col('Decade') >= 1960) # col was necessary to read new column Decade
ele.groupBy('Decade').count().show()

+------+-----+
|Decade|count|
+------+-----+
|  1990|  345|
|  1960|  154|
|  1970|  244|
|  1980|  607|
+------+-----+



In [ ]:
# 12. Find the number of movies released each year.
df.groupBy('Year').count().show()

+----+-----+
|Year|count|
+----+-----+
|1959|    9|
|1990|  105|
|1975|   23|
|1977|   29|
|1924|    3|
|1974|   23|
|1927|    3|
|1955|   20|
|1978|   18|
|1925|    1|
|1961|   14|
|1942|    6|
|1944|    4|
|1939|   11|
|1952|   10|
|1956|   15|
|1934|    4|
|1988|   96|
|1997|    1|
|1968|   20|
+----+-----+
only showing top 20 rows



In [ ]:

# 13. Find the number of movies released in each year of each genre. Consider only the movies with a length greater than 100 minutes.
df.filter(df.Length > 100).groupBy('Year', 'Genre').count().show()


+----+---------------+-----+
|Year|          Genre|count|
+----+---------------+-----+
|1989|         Action|   10|
|1988|         Comedy|   10|
|1966|          Drama|    6|
|1985|         Comedy|    6|
|1957|        Western|    1|
|1928|          Drama|    2|
|1970|        Western|    2|
|1975|          Drama|    5|
|1969|         Action|    1|
|1940|         Action|    1|
|1939|        Western|    3|
|1987|         Comedy|   13|
|1982|Science Fiction|    1|
|1951|            War|    1|
|1965|          Music|    1|
|1939|         Comedy|    1|
|1964|          Music|    1|
|1987|            War|    1|
|1962|         Action|    1|
|1966|        Mystery|    3|
+----+---------------+-----+
only showing top 20 rows



In [ ]:

# 14. Sort the movie’s release before 1990 by the title.
df.filter(df.Year < 1990).orderBy('Title').show()

+----+------+--------------------+---------------+--------------------+-----------------+-------------------+----------+------+-------------------+
|Year|Length|               Title|          Genre|               Actor|          Actress|           Director|Popularity|Awards|              Image|
+----+------+--------------------+---------------+--------------------+-----------------+-------------------+----------+------+-------------------+
|1968|   139|2001: A Space Ody...|Science Fiction|          DulleaKeir|   TyzackMargaret|     KubrickStanley|        83|    No|   NicholasCage.png|
|1982|    92|             48 Hrs.|         Action|           NolteNick|   O'TooleAnnette|         HillWalter|        67|    No|   NicholasCage.png|
|1963|   138|               8 1/2|          Drama| MastroianniMarcello| CardinaleClaudia|    FelliniFederico|        80|   Yes|   NicholasCage.png|
|1966|    95|A Big Hand for th...|         Comedy|          FondaHenry|   WoodwardJoanne|        CookFielder|   

In [ ]:
from pyspark.sql.functions import length
# 15. Find the movies with long titles. A movie title is considered long if it is greater than 50 alphabets
df.filter(length(df.Title) > 50).show()

+----+------+--------------------+------+----------+--------------+--------+----------+------+----------------+
|Year|Length|               Title| Genre|     Actor|       Actress|Director|Popularity|Awards|           Image|
+----+------+--------------------+------+----------+--------------+--------+----------+------+----------------+
|1979|    90|Fawlty TowersGour...|Comedy|CleeseJohn|ScalesPrunella|    NULL|        46|    No|NicholasCage.png|
+----+------+--------------------+------+----------+--------------+--------+----------+------+----------------+

